In [1]:
!pip install hypertools

In [1]:
import numpy as np
from scipy.linalg import cholesky
from scipy.spatial.distance import squareform
from scipy.stats.stats import pearsonr
import sys
import os
sys.path.append(os.path.abspath('../'))
from timecorr.timecorr import levelup, sliding_window_levelup
import matplotlib.pyplot as plt
from math import log

sliding_window_length = 51
block_length = 1
covariance_num = 1000
noise_level = [0.01,0.1,1,10]
repetitions=1
noise_num = len(noise_level)
time_range=time_len = block_length * covariance_num
activation_num = 6
subject_num = 5
variance = 1000
activations = [[np.random.normal(0,1,[activation_num, time_len]) for i in range(subject_num)] for j in range(noise_num)]
correlations = np.zeros([covariance_num,activation_num,activation_num])
correlation1,correlation2 = np.zeros([activation_num,activation_num]), np.zeros([activation_num,activation_num])


def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def cholesky_ramp_correlation_data():
    global activations, correlations,correlation1,correlation2
    correlations = np.zeros([covariance_num,activation_num,activation_num])
    activations_temp = np.random.normal(0,1,[activation_num, time_len])
    
    while not is_pos_def(correlation1):
        feature_map1 = np.random.normal(0,1,[activation_num,activation_num])
        correlation1 = np.dot(feature_map1,feature_map1.T)
        correlation1 = correlation1/np.max(abs(correlation1))
    while not is_pos_def(correlation2):
        feature_map2 = np.random.normal(0,1,[activation_num,activation_num])
        correlation2 = np.dot(feature_map2,feature_map2.T)
        correlation2 = correlation2/np.max(abs(correlation2))
    for i in range(time_len/2):
        cov_temp = (time_len-i)*0.5*(np.log(1+correlation1+1e-5) - np.log(1-correlation1+1e-5))/float(time_len)+i*0.5*(np.log(1+correlation2+1e-5) - np.log(1-correlation2+1e-5))/float(time_len)
        correlations[i] =  (np.exp(2*cov_temp) - 1)/(np.exp(2*cov_temp) + 1)
        activations_temp[:,i] = np.dot(cholesky(correlations[i]),activations_temp[:,i])
    for i in range(time_len/2):
        correlations[time_len/2+i] =  correlations[time_len/2-i-1]
        activations_temp[:,time_len/2+i] = np.dot(cholesky(correlations[time_len/2+i]),activations_temp[:,time_len/2+i])
    for i in range(noise_num):
        activations[i]=[(activations_temp+np.random.normal(0,noise_level[i],[activation_num, time_len])).T for s in range(subject_num)]

level1_correlations,level2_correlations = np.zeros([noise_num, time_len-sliding_window_length+1]),np.zeros([noise_num, time_len-2*sliding_window_length+2])
level1_MSE, level2_MSE = np.zeros([noise_num, time_len-sliding_window_length+1]),np.zeros([noise_num, time_len-2*sliding_window_length+2])
for i in range(repetitions):
    cholesky_ramp_correlation_data()
    for n in range(noise_num):
        timecorr_level1 = levelup(activations[n],"across")
        temp = levelup(activations[n],"within")
        for z in range(len(temp)):
            temp[z] = temp[z].T
        timecorr_level2 = levelup(temp,"across")
        
        sliding_window_level1 = sliding_window_levelup(activations[n],"across")
        temp = sliding_window_levelup(activations[n],"within")
        for z in range(len(temp)):
            temp[z] = temp[z].T
        sliding_window_level2 = sliding_window_levelup(temp,"across")
        
        for t in range(time_len-sliding_window_length+1):
            otc = pearsonr(timecorr_level1[t+sliding_window_length/2], sliding_window_level1[t])[0]
            level1_correlations[n,t] += 0.5 * (log(1+otc) - log(1-otc))
            level1_MSE[n,t] += np.sum(np.square(timecorr_level1[t+sliding_window_length/2]-sliding_window_level1[t]))
        
        for t in range(time_len-2*sliding_window_length+2):
            otc = pearsonr(timecorr_level2[t+sliding_window_length], sliding_window_level2[t])[0]
            level2_correlations[n,t] += 0.5 * (log(1+otc) - log(1-otc))
            level2_MSE[n,t] += np.sum(np.square(timecorr_level2[t+sliding_window_length]-sliding_window_level2[t]))

level1_correlations /= repetitions
level1_correlations =  (np.exp(2*level1_correlations) - 1)/(np.exp(2*level1_correlations) + 1)

level2_correlations /= repetitions
level2_correlations =  (np.exp(2*level2_correlations) - 1)/(np.exp(2*level2_correlations) + 1)

level1_MSE/= repetitions
level2_MSE/= repetitions

f, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, sharey='row', figsize=(28,24))
plt.subplots_adjust(top=0.95)
plt.suptitle("10 discrete correlations, each occupying a block of 100 timepoints",fontsize=20)
a1.set_ylabel("timecorr vs sliding window level 1 correlation")
a2.set_ylabel("timecorr vs sliding window level 1 MSE")
a3.set_ylabel("timecorr vs sliding window level 2 correlation")
a4.set_ylabel("timecorr vs sliding window level 2 MSE")

for v in range(noise_num): 
    a1.plot(range(sliding_window_length/2,time_len-sliding_window_length/2),level1_correlations[v],c='C'+str(i),alpha=1,linestyle='-', label="noise level "+str(noise_level[v]))
    a2.plot(range(sliding_window_length/2,time_len-sliding_window_length/2),level1_MSE[v],c='C'+str(i),alpha=1,linestyle='-', label="noise level "+str(noise_level[v]))
    
    a3.plot(range(sliding_window_length-1,time_len-sliding_window_length+1),level2_correlations[v],c='r',alpha=1,linestyle='-', label="noise level "+str(noise_level[v]))
    a4.plot(range(sliding_window_length-1,time_len-sliding_window_length+1),level2_MSE[v],c='C1',alpha=0.5,linestyle='--', label="noise level "+str(noise_level[v]))
    
a1.set_xlabel("time")
a2.set_xlabel("time")
a3.set_xlabel("time")
a4.set_xlabel("time")
a1.legend(bbox_to_anchor=(2, 1), loc=0)
a2.legend(bbox_to_anchor=(2, 1), loc=0)
a3.legend(bbox_to_anchor=(2, 1), loc=0)
a4.legend(bbox_to_anchor=(2, 1), loc=0)
f.subplots_adjust(hspace=0.2)
plt.show()

across
(1000, (15,))
within
(5, (1000, 15))


ValueError: Found array with dim 3. Estimator expected <= 2.